In [1]:
import pandas as pd
import pickle
import re
import numpy as np
from tqdm import tqdm
from collections import defaultdict 
import pickle
import seaborn as sns

In [2]:
import os
from os import listdir
import sys

In [3]:
file_dir = '../'

data_dir = file_dir + 'test_data/'
raw_data_dir = data_dir + 'raw/'
result_dir = data_dir + 'results/'

In [4]:
sys.path.insert(1, '../foodparser/')

from foodparser import *

[nltk_data] Downloading package words to /Users/joey-hou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joey-hou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# fp = FoodParser()
# fp.initialization()

In [7]:
# fp.run_setup()

In [9]:
# !pip install pyspellchecker
from spellchecker import SpellChecker

import re
import string
import nltk

from nltk.corpus import stopwords, words
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wnl = WordNetLemmatizer()

nltk.download('averaged_perceptron_tagger')

# Load stop words
stop_words = stopwords.words('english')
# Updated by Katherine August 23rd 2020
stop_words.remove('out') # since pre work out is a valid beverage name
stop_words.remove('no')
stop_words.remove('not')

wordlist = words.words()

[nltk_data] Downloading package words to /Users/joey-hou/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joey-hou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joey-
[nltk_data]     hou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joey-hou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Data Formatting / Making test file

## Entire training set

In [10]:
public_all_pickle_file = open(raw_data_dir + 'public_0924_basline_usable_expanded.pickle', 'rb')
public_all = pickle.load(public_all_pickle_file)
public_all.head(2)

,ID,unique_code,research_info_id,desc_text,food_type,original_logtime,original_logtime_notz,day,local_time,time,week_from_start,year
281,282,alqt15090005,150,Water,w,09-09-2015 8:20PM JST,2015-09-09 20:20:00,2015-09-09,20.333333,20:20:00,1,2015
282,283,alqt15090005,150,Soup,f,09-09-2015 10:38PM JST,2015-09-09 22:38:00,2015-09-09,22.633333,22:38:00,1,2015


In [11]:
# test_data = public_all.sample(2000)[['desc_text', 'food_type']].copy().reset_index(drop = True)
# test_data.shape

In [12]:
# test_data.to_csv(raw_data_dir + 'test_data.csv', index = False)

In [13]:
# public_all.reset_index(drop = True).to_csv(raw_data_dir + 'all_data.csv', index = False)

In [14]:
public_all.shape

(1653926, 12)

In [15]:
all_loggings = public_all.desc_text.values
all_entries = []
for l in all_loggings:
    if ',' in l:
        for entry in l.split(','):
            all_entries.append(entry.strip())
    else:
        all_entries.append(l)

In [16]:
len(all_entries)

2399626

In [17]:
all_entries_length = ([len(i.split()) for i in all_entries])

In [18]:
sum(np.array(all_entries_length) > 5)

21325

## Additional training texts

In [19]:
all_unknown_phrases_df = pd.read_csv(result_dir + 'all_unknown_phrases.csv').dropna()
all_unknown_phrases_df.shape

(764077, 1)

In [20]:
all_unknown_phrases_df.head(2)

,unknown_phrases
0,isotonic drink
1,sashimi


# Building improved food annotations

In [21]:
fp = FoodParser()
fp.initialization()

## Load known grams

In [22]:
known_grams = pd.read_csv('../foodparser/data/combined_gram_set.csv')
known_grams.head(2)

,gram_key,food_type,gram_type,tag1,tag2,tag3,tag4
0,chilli,f,1,NaN,NaN,NaN,NaN
1,chile,f,1,NaN,NaN,NaN,NaN


In [23]:
known_grams.tail(2)

,gram_key,food_type,gram_type,tag1,tag2,tag3,tag4
1100,curcumin,m,1,NaN,NaN,NaN,NaN
1101,apple cider,b,2,NaN,NaN,NaN,NaN


In [24]:
known_grams_lst = list(known_grams.gram_key.values)
known_grams_set = set(known_grams_lst)

## Training Data Pre-processing

In [25]:
my_stop_words = ['oc', 'amp', 'homemade', 'food', 'foodporn', 'v', 'x', 'recipe', 'best', 'il', 'ever',
                'time', 'attempt', 'first', 'ate', 'made', 'home', 'today', 'make', 'friend', 'local',
                'new', 'day', 'birthday', 'like', 'amazing', 'de', 'happy', 'year', 'plate', 'video', 
                'cooked', 'dish', 'house', 'os', 'tried', 'super', 'perfect', 'way', 'delicious', 
                'night', 'last', 'eating', 'know', 
                 'mg', 'oz', 'and', 'w', 'x', 'n', 'ml'
                 'handful']

# Function for removing numbers
def handle_numbers(text):
    clean_text = text
    clean_text = re.sub('[0-9]+\.[0-9]+', '' , clean_text)
    clean_text = re.sub('[0-9]+', '' , clean_text)
    clean_text = re.sub('\s\s+', ' ', clean_text)
    return clean_text

# Function for removing punctuation
def drop_punc(text):
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return clean_text

# Remove normal stopwords
def remove_stop(my_text):
    text_list = my_text.split()
    return ' '.join([word for word in text_list if word not in stop_words])

def remove_my_stop(text):
    text_list = text.split()
    return ' '.join([word for word in text_list if word not in my_stop_words])

def remove_plural(w):
    return wnl.lemmatize(w)

def pre_processing(text):
    text = text.lower()
    text = remove_my_stop(remove_stop(handle_numbers(drop_punc(text)))).strip()
    return ' '.join([remove_plural(w) for w in text.split()])

In [26]:
all_text = public_all.desc_text.values

In [27]:
all_text_cleaned = [pre_processing(tx) for tx in tqdm(all_text)]

100%|██████████| 1653926/1653926 [00:52<00:00, 31748.94it/s]


In [28]:
len(all_text_cleaned)

1653926

In [29]:
all_text_cleaned[:3]

['water', 'soup', 'yogurt']

In [30]:
all_food_type = list(public_all.food_type.values)
all_food_type[:3]

['w', 'f', 'b']

In [31]:
training_texts = list(zip(all_text_cleaned, all_food_type))
training_texts[:3]

[('water', 'w'), ('soup', 'f'), ('yogurt', 'b')]

## Implement food mining

In [32]:
list(known_grams_set)[:3]

['chia squeeze', 'greek yogurt granola', 'vitamin k']

In [33]:
def handle_single_phrase(ph, curr_phrase_dict, known_phrase_dict, known_sub_phrases_set):
    # Check known phrases list
    try:
        known_phrase_dict[ph] += 1
        return 0
    except:
        pass
    # Add to current phrase list if its not a subphrase
    if ph not in known_sub_phrases_set:
        curr_phrase_dict[ph] += 1
        return 1
    return 0

def prepare_phrase_base(known_grams_set):
    all_known_phrases_lst = []
    for file in ['quad_grams.csv', 'bi_grams.csv', 'uni_grams.csv', 'tri_grams.csv', 'penta_grams.csv']:
        curr_df = pd.read_csv('../foodparser/data/annotations/' + file)
        curr_phrases = curr_df.key.values
        all_known_phrases_lst += list(curr_phrases)
    all_known_sub_phrases_set = set(all_known_phrases_lst)
    return all_known_sub_phrases_set

def run_training(known_grams_set, training_texts):
    '''
    Input format:
        - known_grams_set: set object, e.g. {'water', 'preworkout', ...}
        - training_texts: list object, e.g. [('water', 'w'), ('preworkout', 'b'), ...]
    '''
    
    # Make a spellChecker instance
    # spell = SpellChecker()

    # Known phrases 
    known_phrase_dict = dict(zip(known_grams_set, [0] * len(list(known_grams_set))))
    known_sub_phrases_set = prepare_phrase_base(known_grams_set)
    
    # Scanning through the documents
    curr_phrase_dict = defaultdict(lambda : 0)

    for doc_pair in tqdm(training_texts):
        if len(doc_pair) == 2:
            doc, food_type = doc_pair
        else:
            doc = doc_pair
        tokens = nltk.word_tokenize(doc)
        
        # Uni-grams
        [handle_single_phrase(ph, curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in tokens]
        
        # N-grams
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 2)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 3)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 4)]
        [handle_single_phrase(' '.join(list(ph)), curr_phrase_dict, known_phrase_dict, known_sub_phrases_set) for ph in nltk.ngrams(tokens, 5)]
        
    return dict(known_phrase_dict), dict(curr_phrase_dict)

In [34]:
known_phrase_dict, curr_phrase_dict = run_training(known_grams_set, training_texts)
add_known_phrase_dict, add_curr_phrase_dict = run_training(known_grams_set, all_unknown_phrases_df.unknown_phrases.values)

100%|██████████| 764077/764077 [01:08<00:00, 11220.47it/s]


In [35]:
curr_phrase_dict = dict(curr_phrase_dict)
known_phrase_dict = dict(known_phrase_dict)

sorted_old_phrases = sorted(known_phrase_dict.items(), key = lambda x: x[1], reverse = True)
sorted_new_phrases = sorted(curr_phrase_dict.items(), key = lambda x: x[1], reverse = True)

len(sorted_old_phrases), len(sorted_new_phrases)

(1102, 2181428)

In [36]:
add_curr_phrase_dict = dict(add_curr_phrase_dict)
add_known_phrase_dict = dict(add_known_phrase_dict)

sorted_old_phrases_add = sorted(add_known_phrase_dict.items(), key = lambda x: x[1], reverse = True)
sorted_new_phrases_add = sorted(add_curr_phrase_dict.items(), key = lambda x: x[1], reverse = True)

In [37]:
# sorted_new_phrases_add

## Filter-off "sub-grams"
### Construct sub-gram bank

In [38]:
all_sub_grams = []
for ph in known_phrase_dict.keys():
    tokens = nltk.word_tokenize(ph)
    for tok in tokens:
        all_sub_grams.append((tok, known_phrase_dict[ph]))
    if len(tokens) == 1:
        continue
        
    for n in [2, 3, 4, 5]:
        for sub_gram in nltk.ngrams(tokens, n):
            all_sub_grams.append((' '.join(sub_gram), known_phrase_dict[ph]))

sub_gram_freq = defaultdict(lambda : 0)
# Accumulate frequency
for pair in all_sub_grams:
    sub_gram_freq[pair[0]] += pair[1]

In [39]:
sub_gram_freq['apple cider']

6030

### Filter off them from new frequency table

In [40]:
filtered_new_phrases = []
for pair in sorted_new_phrases:
    if pair[0] in sub_gram_freq.keys():
        filtered_new_phrases.append((pair[0], pair[1] - sub_gram_freq[pair[0]]))
    else:
        filtered_new_phrases.append(pair)

In [41]:
sorted_filtered_new_phrases = sorted(filtered_new_phrases, key = lambda x: x[1], reverse = True)
sorted_filtered_new_phrases[:10]

[('ml', 7508),
 ('g', 7277),
 ('free', 7261),
 ('no', 5634),
 ('b', 4851),
 ('med', 4819),
 ('crisp', 3353),
 ('stevia', 3255),
 ('unsweetened', 3079),
 ('kefir', 2986)]

In [42]:
len([i for i in sorted_filtered_new_phrases if i[1] > 250 and i[1] < 750])

1035

In [43]:
selected_phrases = [i for i in sorted_filtered_new_phrases if i[1] > 250]

key = [pair[0] for pair in selected_phrases]
gram_type = [len(nltk.word_tokenize(pair[0])) for pair in selected_phrases]
count = [pair[1] for pair in selected_phrases]

additional_phrases = pd.DataFrame({
    'key': key,
    'gram_type': gram_type,
    'count': count
}).sort_values(by = 'gram_type', ascending = False)

additional_phrases.to_csv(result_dir + 'additional_phrases.csv', index = False)

In [44]:
# selected_phrases

In [51]:
# combined_df = pd.read_csv('../foodparser/data/combined_gram_set.csv')
# combined_df.shape

In [52]:
# combined_df.query('food_type != "n"').reset_index(drop = True).to_csv('../foodparser/data/combined_gram_set.csv', index = False)


In [55]:
# combined_df.query('food_type != "n"')

In [91]:
# dict(sorted_filtered_new_phrases)['apple']
50 * 15

750

## Frequency cutoff to maximize entry rate

In [35]:
# sorted_new_phrases

In [36]:
'egg plant' in [pair[0] for pair in sorted_new_phrases]

True

In [37]:
# Ignoring all the phrases with less than 10 apperances
all_phrase_freq = [pair for pair in sorted_old_phrases + sorted_new_phrases if pair[1] > 10]
len(all_phrase_freq)

60951

In [38]:
all_phrase_freq[:3]

[('water', 397700), ('coffee', 142167), ('tea', 83643)]

In [39]:
all_phrase_freq_w_length = [(pair[0], pair[1], len(pair[0].split())) for pair in all_phrase_freq]
all_phrase_freq_w_length[:3]

[('water', 397700, 1), ('coffee', 142167, 1), ('tea', 83643, 1)]

In [40]:
all_unigram_freq_w_length = [pair for pair in all_phrase_freq_w_length if pair[2] == 1]
len(all_unigram_freq_w_length)

8286

In [41]:
all_unigram_freq_w_length[:3]

[('water', 397700, 1), ('coffee', 142167, 1), ('tea', 83643, 1)]

In [42]:
sorted_all_phrase_freq_w_length = sorted(all_phrase_freq_w_length, key = lambda x: x[2], reverse = True)
sorted_all_phrase_freq_w_length[:3]

[('cold brew coffee almond milk', 11, 5),
 ('olive oil apple cider vinegar', 65, 5),
 ('whole wheat bread peanut butter', 58, 5)]

In [43]:
len(training_texts)

1653926

In [44]:
def calc_entry_rate(gram_set, training_texts):
    tmp_lst = [pair[0] for pair in training_texts]
    prev_length = len(tmp_lst)
    
    total_entry_matched = 0
    for gram in tqdm(gram_set):
        tmp_lst =  [entry for entry in tmp_lst if gram not in entry.split()]
        curr_length = len(tmp_lst)
        total_entry_matched += (prev_length - curr_length)
        prev_length = curr_length
    return total_entry_matched / len(training_texts)

In [45]:
tmp_grams = [pair[0] for pair in all_unigram_freq_w_length[:500]]
calc_entry_rate(tmp_grams, training_texts)

100%|██████████| 500/500 [00:24<00:00, 20.47it/s]


0.9183627320690285

In [46]:
all_unigram_freq_w_length[:3]

[('water', 397700, 1), ('coffee', 142167, 1), ('tea', 83643, 1)]

In [47]:
# needed_token_bank = set([pair[0] for pair in all_unigram_freq_w_length[:500]])
# len(list(needed_token_bank))

In [48]:
# needed_phrases = []
# unique_tokens = defaultdict(lambda : 0)
# for pair in sorted_all_phrase_freq_w_length:
#     found = False
#     for token in pair[0].split():
#         if token in needed_token_bank:
#             unique_tokens[token] += 1
#             found = True
#     if found and pair[0] not in known_grams_set:
#         needed_phrases.append(pair)
#     if len(dict(unique_tokens).keys()) >= 500:
#         break

In [49]:
# len(unique_tokens)

In [50]:
# len(needed_phrases)

In [148]:
# tmp_knowledge_base = list(prepare_phrase_base(known_grams_set))
# all_token = []
# for i in tmp_knowledge_base:
#     for token in i.split():
#         if token not in all_token:
#             all_token.append(token)

In [51]:
# len(all_token)

In [52]:
# all_needed_phrases = [pair[0] for pair in needed_phrases]
# mask = [True] * len(all_needed_phrases)
# for idx in tqdm(range(len(all_needed_phrases) - 1, -1, -1)):
#     curr_ph = all_needed_phrases[idx]
#     for ph in all_needed_phrases:
#         if ph != curr_ph:
#             if curr_ph + ' ' in ph or ' ' + curr_ph in ph:
#                 mask[idx] = False
#                 break

In [53]:
# len(np.array(all_needed_phrases)[mask])

In [114]:
# all_known_phrases_lst = []
# for file in ['quad_grams.csv', 'bi_grams.csv', 'uni_grams.csv', 'tri_grams.csv', 'penta_grams.csv']:
#     curr_df = pd.read_csv('../resources/' + file)
#     curr_phrases = curr_df.key.values
#     all_known_phrases_lst += list(curr_phrases)

In [115]:
# len(all_known_phrases_lst)

In [49]:
key = [pair[0] for pair in sorted_items]
gram_type = [len(nltk.word_tokenize(pair[0])) for pair in sorted_items]
count = [pair[1] for pair in sorted_items]

additional_phrases = pd.DataFrame({
    'key': key,
    'gram_type': gram_type,
    'count': count
}).sort_values(by = 'gram_type', ascending = False)

additional_phrases.to_csv('../data/result/additional_phrases.csv', )

In [36]:
key[:3], gram_type[:3], count[:3]

(['ml', 'herbal', 'b'], [1, 1, 1], [12170, 8492, 7861])

## Refinment

In [51]:
curr_phrase_bank = [pair[0] for pair in sorted_items]
curr_phrase_bank[:3], len(curr_phrase_bank)

(['ml', 'herbal', 'b'], 1731)

In [52]:
print('=> Length of curr_phrase_bank:', len(curr_phrase_bank))
    
tmp_mask = np.array([True] * len(curr_phrase_bank))
curr_phrase_bank = np.array(curr_phrase_bank)
    
curr_idx = 0
for ph in tqdm(curr_phrase_bank):
    tokens = nltk.word_tokenize(ph)
    curr_gram_length = len(tokens)
    for other_ph in curr_phrase_bank:
        if ph == other_ph:
            continue
            
        other_ph_tokens = nltk.word_tokenize(other_ph)
        if len(other_ph_tokens) < curr_gram_length:
            continue
            
        n_grams = [' '.join(gram) for gram in nltk.ngrams(other_ph_tokens, curr_gram_length)]
        if ph in n_grams:
            tmp_mask[curr_idx] = False
            break
    curr_idx += 1
if sum(tmp_mask) == len(tmp_mask):
    print('    => No Change to current phrase bank!')
#     break
curr_phrase_bank = curr_phrase_bank[tmp_mask]
print(len(curr_phrase_bank))


  0%|          | 4/1731 [00:00<01:04, 26.85it/s]

=> Length of curr_phrase_bank: 1731


100%|██████████| 1731/1731 [03:24<00:00,  8.47it/s]


SyntaxError: 'break' outside loop (<ipython-input-52-2270ebb5ba5d>, line 25)

In [54]:
curr_phrase_bank = curr_phrase_bank[tmp_mask]

print(len(curr_phrase_bank))

1413
